In [ ]:
import os
from du_astro_utils import calibration, photometry, utils
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from scipy.ndimage import median_filter
from tqdm import tqdm

## Reduce Images

In [ ]:
target_type = "galaxy_cluster"
if target_type == "asteroid":
    data_dir = os.path.join(utils.C2PU_DATA_DIR, utils.DIR_PHOTOM, utils.DIR_ASTER)
elif target_type == "exoplanet":
    data_dir = os.path.join(utils.C2PU_DATA_DIR, utils.DIR_PHOTOM, utils.DIR_EXPLTS)
elif target_type == "variable_star":
    data_dir = os.path.join(utils.C2PU_DATA_DIR, utils.DIR_PHOTOM, utils.DIR_VARSTARS)
elif target_type == "galaxy_cluster":
    data_dir = os.path.join(utils.C2PU_DATA_DIR, utils.DIR_PHOTOM, utils.DIR_GALCLUST)
elif target_type == "cluster":
    data_dir = os.path.join(utils.C2PU_DATA_DIR, utils.DIR_PHOTOM, utils.DIR_CLUSTERS)
os.listdir(data_dir)

In [ ]:
for ix, ddir in enumerate(os.listdir(data_dir)):
    subdata_dir = os.path.join(data_dir, ddir)
    if os.path.isdir(subdata_dir):
        list_fits = [im for im in sorted(os.listdir(subdata_dir)) if ".fits" in im]
        list_fits = sorted(list_fits)
        print(f"{ix} - {subdata_dir} : {len(list_fits)} files")

In [ ]:
achoice = os.listdir(data_dir)[8]
subdata_dir = os.path.join(data_dir, achoice)
list_fits = sorted([im for im in sorted(os.listdir(subdata_dir)) if ".fits" in im])
list_fits
im0 = os.path.join(subdata_dir, list_fits[0])
with fits.open(im0) as hdul:
    hdr = hdul[0].header
print(hdr.keys)
hdr.get("FOCPOS")

In [ ]:
for idx in [7]:  # range(6, 10):
    achoice = os.listdir(data_dir)[idx]
    reduced = False
    aligned = False
    subdata_dir = os.path.join(data_dir, achoice)
    if reduced:
        subdata_dir = os.path.join(subdata_dir, "REDUCED")
    if aligned:
        subdata_dir = os.path.join(subdata_dir, "aligned")
    list_fits = [im for im in sorted(os.listdir(subdata_dir)) if ".fits" in im]
    list_fits = sorted(list_fits)

    rerun = False
    test_mode = False  # Only runs on a few images, prints the tables at each step and does not write files.
    # dict_of_dict = {}

    if test_mode:
        list_fits = list_fits[:10]
    for loc, scimage in enumerate(tqdm(list_fits)):
        fits_sci_image = os.path.join(subdata_dir, scimage)
        fits_sci_image = os.path.abspath(fits_sci_image)
        if os.path.isfile(fits_sci_image):
            # Get image directory, failename and extension
            sc_im_dir = os.path.abspath(os.path.dirname(fits_sci_image))
            sc_im_name, sc_im_ext = os.path.splitext(os.path.basename(fits_sci_image))

            bias_dir, darks_dir, flats_dir = utils.get_calib_dirs_photometry(fits_sci_image)
            # print(bias_dir, darks_dir, flats_dir)

            # Get information from FITS header
            # sc_date, sc_scope, sc_cam, sc_filter, sc_focus, sc_expos, sc_x, sc_y = calibration.get_infos_from_image(fits_sci_image, verbose=True)
            # print(sc_date, sc_filter, sc_focus, sc_expos)
            # flats_list = calibration.load_flat_frames(flats_dir, sc_date, sc_cam, sc_focus, sc_filter, sc_x, sc_y, verbose=True)
            # for flat in flats_list:
            #    with fits.open(flat) as hdul:
            #        hdr = hdul[0].header
            #        print(hdr.get('FOCPOS'), hdr.get('INSTFILT'))

            # Run calibration
            # dico_calib = calibration.dedark_sci_image(fits_sci_image, override_date_check=True, max_days=7,\
            #                                          overwrite=rerun, verbose=False, write_tmp=test_mode,\
            #                                          overwrite_calibs=False)
            calibration.reduce_sci_image(fits_sci_image, darks_dir, flats_dir, bias_dir, override_date_check=False, max_days=10, overwrite=rerun, verbose=False, write_tmp=test_mode, overwrite_calibs=(rerun and loc == 0))
            # dict_of_dict.update({sc_im_name: dico_calib})

__If not done, plate-solve and align within AIJ.__

## Stacking images

In [ ]:
data_dir = os.path.join(utils.C2PU_RES_DIR, utils.DIR_PHOTOM, utils.DIR_GALCLUST)

In [ ]:
reduced = True
aligned = True
for ix, ddir in enumerate(os.listdir(data_dir)):
    subdata_dir = os.path.join(data_dir, ddir)
    if reduced:
        subdata_dir = os.path.join(subdata_dir, "REDUCED")
    if aligned:
        subdata_dir = os.path.join(subdata_dir, "aligned")
    if os.path.isdir(subdata_dir):
        list_fits = [im for im in sorted(os.listdir(subdata_dir)) if "REDUCED.fits" in im]
        list_fits = sorted(list_fits)
        print(f"{ix} - {subdata_dir} : {len(list_fits)} files")
    else:
        print(f"{subdata_dir} : not a directory.")

In [ ]:
dirchoice = os.listdir(data_dir)[7]
subdata_dir = os.path.join(data_dir, dirchoice)
if reduced:
    subdata_dir = os.path.join(subdata_dir, "REDUCED")
if aligned:
    subdata_dir = os.path.join(subdata_dir, "aligned")
if os.path.isdir(subdata_dir):
    list_fits = [im for im in sorted(os.listdir(subdata_dir)) if "REDUCED.fits" in im]
    list_fits = sorted(list_fits)

In [ ]:
list_fits

In [ ]:
manual_stack = False
do_stack = True
# combtype='MEDIAN' # SUM ou AVERAGE ou MEDIAN
center_type = "MOST"
reduced = True
aligned = True
if do_stack:
    for combtype in tqdm(["SUM", "MEDIAN", "AVERAGE"]):
        for filt in tqdm(["g", "r", "i"]):  # tqdm(['g', 'r', 'i', 'z']):
            exp = 120  # exp = 60 if filt=='z' else 30 if filt=='i' else 15 # 10
            dirchoice = os.listdir(data_dir)[7]  # os.listdir(data_dir)[9] if filt=='z' else os.listdir(data_dir)[8]
            subdata_dir = os.path.join(data_dir, dirchoice)
            if reduced:
                subdata_dir = os.path.join(subdata_dir, "REDUCED")
            if aligned:
                subdata_dir = os.path.join(subdata_dir, "aligned")
            rel_data_dir = os.path.relpath(subdata_dir)

            if not manual_stack:
                outname = f"coadd_{dirchoice}_SDSS{filt}p_{exp:04d}s_RED_SW_{combtype}_{center_type}.fits"
                if not os.path.isfile(outname):
                    swarp_cmd = f"swarp {rel_data_dir}/*SDSS{filt}*{exp:04d}s*REDUCED.fits -c default.swarp -IMAGEOUT_NAME {outname} -COMBINE_TYPE {combtype} -CENTER_TYPE {center_type} -VERBOSE_TYPE QUIET"
                    os.system(swarp_cmd)
            else:
                if os.path.isdir(rel_data_dir):
                    list_fits = [im for im in sorted(os.listdir(rel_data_dir)) if "REDUCED.fits" in im]
                    list_fits = sorted(list_fits)
                outname = f"coadd_{dirchoice}_SDSS{filt}p_{exp:04d}s_RED_np{combtype}.fits"
                SDSS_imgs = [os.path.join(rel_data_dir, im) for im in list_fits if (f"SDSS{filt}" in im and f"{exp:04d}s" in im)]
                if len(SDSS_imgs) > 0:
                    if not os.path.isfile(outname):
                        with fits.open(SDSS_imgs[0]) as hdul:
                            ref_hdu = hdul[0].copy()
                            SDSS_stack = np.empty((len(SDSS_imgs), *hdul[0].data.shape))
                        for loc, fits_img in enumerate(SDSS_imgs):
                            with fits.open(fits_img) as hdul:
                                img_data = hdul[0].data
                            SDSS_stack[loc, :, :] = img_data
                            # mean, med, sigma = sigma_clipped_stats(img_data, sigma=3)
                            # plt.imshow(img_data, cmap='gray', vmin=med-5*sigma, vmax=med+5*sigma)
                            # plt.colorbar()
                            # plt.show()
                        if combtype == "AVERAGE":
                            coadd_im = np.mean(SDSS_stack, axis=0)
                        elif combtype == "SUM":
                            coadd_im = np.sum(SDSS_stack, axis=0)
                        else:  ## defaults to median
                            coadd_im = np.median(SDSS_stack, axis=0)
                        ref_hdu.data = coadd_im
                        ref_hdu.writeto(outname, overwrite=True)

            if os.path.isfile(outname):
                with fits.open(outname) as hdul:
                    img_data = hdul[0].data
                mean, med, sigma = sigma_clipped_stats(img_data, sigma=3)
                plt.imshow(img_data, cmap="gray", vmin=med + 2 * sigma, vmax=med + 5 * sigma)
                # plt.colorbar()
                plt.title(outname)
                plt.show()

In [ ]:
if True:
    for combtype in ["SUM", "MEDIAN", "AVERAGE"]:
        for filt in tqdm(["g", "r", "i"]):  # tqdm(['g', 'r', 'i', 'z']):
            exp = 120  # exp = 60 if filt=='z' else 30 if filt=='i' else 15 # 10
            dirchoice = os.listdir(data_dir)[7]  # os.listdir(data_dir)[9] if filt=='z' else os.listdir(data_dir)[8]
            if not manual_stack:
                stack_name = f"coadd_{dirchoice}_SDSS{filt}p_{exp:04d}s_RED_SW_{combtype}_{center_type}.fits"
                lab = f"SDSS{filt}p_{exp:04d}s_RED_SW_{combtype}"
            else:
                stack_name = f"coadd_{dirchoice}_SDSS{filt}p_{exp:04d}s_RED_np{combtype}.fits"
                lab = f"SDSS{filt}p_{exp:04d}s_RED_np{combtype}"
            with fits.open(stack_name) as stack_im:
                hdr = stack_im[0].header
                data = stack_im[0].data
            mean, med, sigma = sigma_clipped_stats(data, sigma=3)
            plt.imshow(data, cmap="gray", vmin=med + 3 * sigma, vmax=med + 17 * sigma)
            # plt.colorbar()
            plt.title(lab)
            plt.show()